In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge,ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [52]:
data = pd.read_csv('../data/kc_house_data.csv')
data.head(20)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,20150415T000000,229500.0,3,1.00,1780,7470,1.0,0,0,...,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,20150312T000000,323000.0,3,2.50,1890,6560,2.0,0,0,...,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


In [53]:
data = data.drop(["date","id"], axis = 1)

In [54]:
data.corr()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
price,1.000000,0.308350,0.525138,0.702035,0.089661,0.256794,0.266369,0.397293,0.036362,0.667434,0.605567,0.323816,0.054012,0.126434,-0.053203,0.307003,0.021626,0.585379,0.082447
bedrooms,0.308350,1.000000,0.515884,0.576671,0.031703,0.175429,-0.006582,0.079532,0.028472,0.356967,0.477600,0.303093,0.154178,0.018841,-0.152668,-0.008931,0.129473,0.391638,0.029244
bathrooms,0.525138,0.515884,1.000000,0.754665,0.087740,0.500653,0.063744,0.187737,-0.124982,0.664983,0.685342,0.283770,0.506019,0.050739,-0.203866,0.024573,0.223042,0.568634,0.087175
sqft_living,0.702035,0.576671,0.754665,1.000000,0.172826,0.353949,0.103818,0.284611,-0.058753,0.762704,0.876597,0.435043,0.318049,0.055363,-0.199430,0.052529,0.240223,0.756420,0.183286
sqft_lot,0.089661,0.031703,0.087740,0.172826,1.000000,-0.005201,0.021604,0.074710,-0.008958,0.113621,0.183512,0.015286,0.053080,0.007644,-0.129574,-0.085683,0.229521,0.144608,0.718557
floors,0.256794,0.175429,0.500653,0.353949,-0.005201,1.000000,0.023698,0.029444,-0.263768,0.458183,0.523885,-0.245705,0.489319,0.006338,-0.059121,0.049614,0.125419,0.279885,-0.011269
waterfront,0.266369,-0.006582,0.063744,0.103818,0.021604,0.023698,1.000000,0.401857,0.016653,0.082775,0.072075,0.080588,-0.026161,0.092885,0.030285,-0.014274,-0.041910,0.086463,0.030703
view,0.397293,0.079532,0.187737,0.284611,0.074710,0.029444,0.401857,1.000000,0.045990,0.251321,0.167649,0.276947,-0.053440,0.103917,0.084827,0.006157,-0.078400,0.280439,0.072575
condition,0.036362,0.028472,-0.124982,-0.058753,-0.008958,-0.263768,0.016653,0.045990,1.000000,-0.144674,-0.158214,0.174105,-0.361417,-0.060618,0.003026,-0.014941,-0.106500,-0.092824,-0.003406
grade,0.667434,0.356967,0.664983,0.762704,0.113621,0.458183,0.082775,0.251321,-0.144674,1.000000,0.755923,0.168392,0.446963,0.014414,-0.184862,0.114084,0.198372,0.713202,0.119248


In [55]:
y = data["price"]
X = data.drop(["price"], axis=1)

In [56]:
y.head(15)

0      221900.0
1      538000.0
2      180000.0
3      604000.0
4      510000.0
5     1225000.0
6      257500.0
7      291850.0
8      229500.0
9      323000.0
10     662500.0
11     468000.0
12     310000.0
13     400000.0
14     530000.0
Name: price, dtype: float64

In [57]:
X.head(15)

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,4,4.50,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,3,1.00,1780,7470,1.0,0,0,3,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3,2.50,1890,6560,2.0,0,0,3,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [59]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((17290, 18), (17290,), (4323, 18), (4323,))

In [60]:
data.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [61]:
data.count()

price            21613
bedrooms         21613
bathrooms        21613
sqft_living      21613
sqft_lot         21613
floors           21613
waterfront       21613
view             21613
condition        21613
grade            21613
sqft_above       21613
sqft_basement    21613
yr_built         21613
yr_renovated     21613
zipcode          21613
lat              21613
long             21613
sqft_living15    21613
sqft_lot15       21613
dtype: int64

In [62]:
lr = LinearRegression().fit(X_train, y_train)

In [63]:
lr.predict(X_test)

array([605795.39521977, 472256.37712684, 536093.74401894, ...,
       683155.31060432, 556491.87503047, 572362.02987698])

In [64]:
y_pred = lr.predict(X_test)
prediction = pd.DataFrame(data=y_pred, columns = ["Lin_multi"])

def MAE(y_test, y_pred):
    f = lambda x,y: np.abs(x-y) 
    return np.sum(list(map(f, y_test, y_pred)))/len(y_test)

def MSE(y_test, y_pred):
    f = lambda x,y: pow(x-y, 2) 
    return np.sum(list(map(f, y_test, y_pred)))/len(y_test)

def RMSE(y_test, y_pred):
    f = lambda x,y: pow(x-y, 2) 
    return sqrt(np.sum(list(map(f, y_test, y_pred)))/len(y_test))

def MAPE(y_test, y_pred):
    f = lambda x,y: np.abs((x-y)/x) 
    return sqrt(np.sum(list(map(f, y_test, y_pred)))/len(y_test))

def R2(MSE, y_test):
    y_mean = np.mean(y_test)
    f = lambda x: pow(x, 2) 
    return 1 - MSE/(np.sum(list(map(f, np.array(y_test) - y_mean)))/len(y_test))
    
MSE_var=MSE(y_test, y_pred)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}\t\t {MAE(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}\t\t {MSE(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}\t {RMSE(y_test, y_pred)}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}\t {MAPE(y_test, y_pred)}')
print(f'R^2: {lr.score(X_test, y_test)}\t\t {R2(MSE_var, y_test)}')
lr.coef_

MAE: 124770.06875113015		 124770.06875113015
MSE: 38232238243.805435		 38232238243.805435
RMSE: 195530.65806621077	 195530.65806621077
MAPE: 0.5032834961091535	 0.5032834961091535
R^2: 0.7020531128794931		 0.7020531128794931


array([-3.47394992e+04,  3.91263055e+04,  1.11908187e+02,  1.58845601e-01,
        8.96477936e+03,  5.95405713e+05,  5.44011700e+04,  2.60532418e+04,
        9.32912699e+04,  6.81432586e+01,  4.37649284e+01, -2.58349776e+03,
        1.89869577e+01, -5.84053176e+02,  6.01626421e+05, -2.04890329e+05,
        2.46320863e+01, -4.50593540e-01])

In [65]:
parameters = {'alpha': np.arange(0, 1, 0.1)}

ridge_optimal = RandomizedSearchCV(Ridge(), parameters).fit(X_train, y_train)
ridge_optimal.best_params_

C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=8.57121e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=1.88463e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=4.5238e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


{'alpha': 0.9}

In [66]:
parameters = {'alpha': np.arange(0, 1, 0.1)}

lasso_optimal = RandomizedSearchCV(Lasso(), parameters).fit(X_train, y_train)
lasso_optimal.best_params_

C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.740e+14, tolerance: 1.837e+11 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model 

{'alpha': 0.9}

In [67]:
parameters = {'alpha': np.arange(0, 1, 0.1)}

elastic_optimal = RandomizedSearchCV(ElasticNet(), parameters).fit(X_train, y_train)
elastic_optimal.best_params_

C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.740e+14, tolerance: 1.837e+11 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model 

{'alpha': 0.0}

In [68]:
ridge = Ridge(alpha = ridge_optimal.best_params_['alpha']).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
prediction.insert(loc= 0 , column='Lin_multi_ridge', value=y_pred)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

MAE: 124754.46601484061
MSE: 38230269333.729485
RMSE: 195525.62321529494
MAPE: 0.5032117278318591
R^2: 0.702068456753017


array([-3.47708074e+04,  3.91300934e+04,  1.11928694e+02,  1.57925936e-01,
        9.04587046e+03,  5.90664695e+05,  5.45921585e+04,  2.60319429e+04,
        9.33546592e+04,  6.81029183e+01,  4.38243242e+01, -2.58690136e+03,
        1.90084240e+01, -5.81840020e+02,  5.99637553e+05, -2.03819186e+05,
        2.46034830e+01, -4.50925490e-01])

In [69]:
lasso = Lasso(alpha = lasso_optimal.best_params_['alpha']).fit(X_train, y_train)
y_pred = lasso.predict(X_test)
prediction.insert(loc= 0 , column='Lin_multi_lasso', value=y_pred)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lasso.score(X_test, y_test)}')

MAE: 124769.36792639908
MSE: 38232249976.56454
RMSE: 195530.68806856006
MAPE: 0.5032797190195785
R^2: 0.702053021445164


C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.239e+14, tolerance: 2.358e+11
  model = cd_fast.enet_coordinate_descent(


In [71]:
elastic = ElasticNet(alpha = elastic_optimal.best_params_['alpha']).fit(X_train, y_train)
y_pred = elastic.predict(X_test)
prediction.insert(loc= 0 , column='Lin_multi_elastic', value=y_pred)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {elastic.score(X_test, y_test)}')

C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(


MAE: 124770.06875113308
MSE: 38232238243.80436
RMSE: 195530.658066208
MAPE: 0.5032834961091601
R^2: 0.7020531128795015


C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.548e+14, tolerance: 2.358e+11 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


In [72]:
prediction

,Lin_multi_elastic,Lin_multi_lasso,Lin_multi_ridge,Lin_multi
0,605795.395220,605798.464658,605450.662680,605795.395220
1,472256.377127,472267.113833,472538.106039,472256.377127
2,536093.744019,536111.339079,536585.361568,536093.744019
3,441655.068215,441647.827657,441468.456458,441655.068215
4,427957.507127,427970.407621,428483.338715,427957.507128
...,...,...,...,...
4318,586180.950041,586174.043007,586064.526451,586180.950041
4319,413703.405641,413685.082825,413418.275358,413703.405641
4320,683155.310604,683154.515973,683049.795469,683155.310604
4321,556491.875031,556480.532661,556266.226789,556491.875030


регрессия с одним предиктором

In [73]:
y_single = data["price"]
x_single = data[["price","sqft_living"]]
x_single

,price,sqft_living
0,221900.0,1180
1,538000.0,2570
2,180000.0,770
3,604000.0,1960
4,510000.0,1680
...,...,...
21608,360000.0,1530
21609,400000.0,2310
21610,402101.0,1020
21611,400000.0,1600


In [74]:
X_single_train, X_single_test, y_single_train, y_single_test = train_test_split(X, y, test_size=0.2)
lr = LinearRegression().fit(X_single_train, y_single_train)
y_single_pred = lr.predict(X_single_test)
prediction.insert(loc= 0 , column='Lin_single', value=y_single_pred)

print(f'MAE: {mean_absolute_error(y_single_test, y_single_pred)}')
print(f'MSE: {mean_squared_error(y_single_test, y_single_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_single_test, y_single_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_single_test, y_single_pred))}')
print(f'R^2: {lr.score(X_single_test, y_single_test)}')

MAE: 125082.62332771102
MSE: 37328588711.75734
RMSE: 193206.0783509601
MAPE: 0.5125722587650134
R^2: 0.6987475323596982


In [75]:
parameters = {'alpha': np.arange(0, 1, 0.1)}

ridge_optimal = RandomizedSearchCV(Ridge(), parameters).fit(X_single_train, y_single_train)
ridge_optimal.best_params_

C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=4.96822e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=4.56063e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=9.10911e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=2.04647e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


{'alpha': 0.6000000000000001}

In [76]:
parameters = {'alpha': np.arange(0, 1, 0.1)}

lasso_optimal = RandomizedSearchCV(Lasso(), parameters).fit(X_single_train, y_single_train)
lasso_optimal.best_params_

C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.848e+14, tolerance: 1.886e+11 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model 

{'alpha': 0.9}

In [77]:
parameters = {'alpha': np.arange(0, 1, 0.1)}

elastic_optimal = RandomizedSearchCV(ElasticNet(), parameters).fit(X_single_train, y_single_train)
elastic_optimal.best_params_

C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.848e+14, tolerance: 1.886e+11 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model 

{'alpha': 0.0}

In [78]:
ridge = Ridge(alpha = ridge_optimal.best_params_['alpha']).fit(X_single_train, y_single_train)
y_pred = ridge.predict(X_single_test)
prediction.insert(loc= 0 , column='Lin_single_ridge', value=y_pred)

print(f'MAE: {mean_absolute_error(y_single_test, y_single_pred)}')
print(f'MSE: {mean_squared_error(y_single_test, y_single_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_single_test, y_single_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_single_test, y_single_pred))}')
print(f'R^2: {ridge.score(X_single_test, y_single_test)}')

MAE: 125082.62332771102
MSE: 37328588711.75734
RMSE: 193206.0783509601
MAPE: 0.5125722587650134
R^2: 0.6987483023273822


In [79]:
lasso = Lasso(alpha =lasso_optimal.best_params_['alpha']).fit(X_single_train, y_single_train)
y_pred = lasso.predict(X_single_test)
prediction.insert(loc= 0 , column='Lin_single_lasso', value=y_pred)

print(f'MAE: {mean_absolute_error(y_single_test, y_single_pred)}')
print(f'MSE: {mean_squared_error(y_single_test, y_single_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_single_test, y_single_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_single_test, y_single_pred))}')
print(f'R^2: {lasso.score(X_single_test, y_single_test)}')

MAE: 125082.62332771102
MSE: 37328588711.75734
RMSE: 193206.0783509601
MAPE: 0.5125722587650134
R^2: 0.6987479788593374


C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.260e+14, tolerance: 2.377e+11
  model = cd_fast.enet_coordinate_descent(


In [80]:
elastic = ElasticNet(alpha =elastic_optimal.best_params_['alpha']).fit(X_single_train, y_single_train)
y_pred = elastic.predict(X_single_test)
prediction.insert(loc= 0 , column='Lin_single_elastic', value=y_pred)

print(f'MAE: {mean_absolute_error(y_single_test, y_single_pred)}')
print(f'MSE: {mean_squared_error(y_single_test, y_single_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_single_test, y_single_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_single_test, y_single_pred))}')
print(f'R^2: {elastic.score(X_single_test, y_single_test)}')

C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(


MAE: 125082.62332771102
MSE: 37328588711.75734
RMSE: 193206.0783509601
MAPE: 0.5125722587650134
R^2: 0.6987475323597183


C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.568e+14, tolerance: 2.377e+11 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


Полиномиальная регрессия

In [81]:
p = PolynomialFeatures(2)

X_p=p.fit_transform(X)
X_p

array([[1.0000000e+00, 3.0000000e+00, 1.0000000e+00, ..., 1.7956000e+06,
        7.5710000e+06, 3.1922500e+07],
       [1.0000000e+00, 3.0000000e+00, 2.2500000e+00, ..., 2.8561000e+06,
        1.2909910e+07, 5.8354321e+07],
       [1.0000000e+00, 2.0000000e+00, 1.0000000e+00, ..., 7.3984000e+06,
        2.1928640e+07, 6.4995844e+07],
       ...,
       [1.0000000e+00, 2.0000000e+00, 7.5000000e-01, ..., 1.0404000e+06,
        2.0471400e+06, 4.0280490e+06],
       [1.0000000e+00, 3.0000000e+00, 2.5000000e+00, ..., 1.9881000e+06,
        1.8146700e+06, 1.6563690e+06],
       [1.0000000e+00, 2.0000000e+00, 7.5000000e-01, ..., 1.0404000e+06,
        1.3841400e+06, 1.8414490e+06]])

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X_p, y, test_size=0.2)

lr2 = LinearRegression().fit(X_train, y_train)

y_pred = lr2.predict(X_test)
prediction.insert(loc= 0 , column='Lin_polinom', value=y_pred)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lr2.score(X_test, y_test)}')

MAE: 101800.87056600786
MSE: 26024724511.689632
RMSE: 161321.80420417333
MAPE: 0.44991733455323346
R^2: 0.8036856889144603


результат стал лучше, попробуем применить регуляризаторы

In [83]:
parameters = {'alpha': np.arange(0, 1, 0.1)}

ridge_optimal = RandomizedSearchCV(Ridge(), parameters).fit(X_train, y_train)
ridge_optimal.best_params_

{'alpha': 0.1}

In [84]:
parameters = {'alpha': np.arange(0, 1, 0.1)}

lasso_optimal = RandomizedSearchCV(Lasso(), parameters).fit(X_train, y_train)
lasso_optimal.best_params_

C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.957e+14, tolerance: 1.845e+11 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model 

{'alpha': 0.9}

In [85]:
lr2 = Lasso(alpha=lasso_optimal.best_params_['alpha']).fit(X_train, y_train)

y_pred = lr2.predict(X_test)
prediction.insert(loc= 0 , column='Lin_polinom_lasso', value=y_pred)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lr2.score(X_test, y_test)}')

MAE: 111056.73977273535
MSE: 31358584732.377174
RMSE: 177083.55296971306
MAPE: 0.46162303625297063
R^2: 0.7634503698362314


C:\Users\1\Jupyter Notebook\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.469e+14, tolerance: 2.340e+11
  model = cd_fast.enet_coordinate_descent(


In [86]:
lr2 = Ridge(alpha=ridge_optimal.best_params_['alpha']).fit(X_train, y_train)

y_pred = lr2.predict(X_test)
prediction.insert(loc= 0 , column='Lin_polinom_ridge', value=y_pred)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lr2.score(X_test, y_test)}')

MAE: 101517.8745043585
MSE: 26025525019.544415
RMSE: 161324.28527516994
MAPE: 0.44842098934006386
R^2: 0.803679650381835


самый точный прогноз - полиномиальная регрессия с риджем

попробуем составить что то вроде ансамбля на основе наших прогнозов, для результатов применим масштабирование

In [87]:
prediction

,Lin_polinom_ridge,Lin_polinom_lasso,Lin_polinom,Lin_single_elastic,Lin_single_lasso,Lin_single_ridge,Lin_single,Lin_multi_elastic,Lin_multi_lasso,Lin_multi_ridge,Lin_multi
0,445628.028324,519288.188602,436284.936531,4.932568e+05,4.932475e+05,4.931605e+05,4.932568e+05,605795.395220,605798.464658,605450.662680,605795.395220
1,448390.635269,477927.186094,437672.892792,1.489182e+06,1.489080e+06,1.486865e+06,1.489182e+06,472256.377127,472267.113833,472538.106039,472256.377127
2,658062.319675,621214.283584,664829.145634,1.239400e+05,1.239466e+05,1.241145e+05,1.239400e+05,536093.744019,536111.339079,536585.361568,536093.744019
3,558451.569061,548036.173933,592659.229961,4.576700e+05,4.576516e+05,4.574767e+05,4.576700e+05,441655.068215,441647.827657,441468.456458,441655.068215
4,333129.080528,338804.576015,338174.500149,8.104006e+05,8.103909e+05,8.102700e+05,8.104006e+05,427957.507127,427970.407621,428483.338715,427957.507128
...,...,...,...,...,...,...,...,...,...,...,...
4318,497680.459782,531974.447777,498948.118629,4.132716e+05,4.132691e+05,4.132385e+05,4.132716e+05,586180.950041,586174.043007,586064.526451,586180.950041
4319,328291.303150,379250.173865,340181.998703,2.258948e+05,2.258901e+05,2.258821e+05,2.258948e+05,413703.405641,413685.082825,413418.275358,413703.405641
4320,738554.314610,846759.718636,741995.616478,1.583469e+06,1.583469e+06,1.583568e+06,1.583469e+06,683155.310604,683154.515973,683049.795469,683155.310604
4321,282273.585289,220072.716432,284752.990543,4.206765e+05,4.206586e+05,4.204892e+05,4.206765e+05,556491.875031,556480.532661,556266.226789,556491.875030


In [91]:
X1_train, X1_test, y1_train, y1_test = train_test_split(prediction, y_test, test_size=0.2)

lr3 = LinearRegression()


lr3.fit(X1_train, y1_train)

y1_pred = lr3.predict(X1_test)

print(f'MAE: {mean_absolute_error(y1_test, y1_pred)}')
print(f'MSE: {mean_squared_error(y1_test, y1_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y1_test, y1_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y1_test, y1_pred))}')
print(f'R^2: {lr3.score(X1_test, y1_test)}')

MAE: 104274.86543699872
MSE: 23431656771.3522
RMSE: 153074.02382949303
MAPE: 0.46423297052472307
R^2: 0.8082002169897631


Этот вариант оказался самым точным, ура!

Сделаем пайплайн первой регрессии

In [89]:
Input = [('minmaxscaler', MinMaxScaler()), ('model', LinearRegression()) ]

pipe = Pipeline(Input)
pipe.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('model', LinearRegression())])

In [90]:
y_pred = pipe.predict(X_test)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {pipe.score(X_test, y_test)}')

MAE: 101346.87078203229
MSE: 25876943622.33856
RMSE: 160863.1207652598
MAPE: 0.44825432102924834
R^2: 0.8048004558919757
